Original code obtained from **[here](https://github.com/intelligent-environments-lab/ProfileClustering)** by Intelligent Environment Lab, UT Austin

In [1]:
# Clear the store before running each time
within_cluster_dist_sum_store = {}

# Built-in libraries
import time
from itertools import product
from math import log
import pickle

# NumPy, SciPy and Pandas
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

# Scikit-Learn
from sklearn.metrics import silhouette_samples, calinski_harabaz_score, silhouette_score
from sklearn.metrics.pairwise import pairwise_distances
# from tslearn.clustering import silhouette_score


# Matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm


In [2]:
def divide(data, labels):
    clusters = set(labels)
    clusters_data = []
    for cluster in clusters:
        clusters_data.append(data[labels == cluster, :])
    return clusters_data

def get_centroids(clusters):
    centroids = []
    for cluster_data in clusters:
        centroids.append(cluster_data.mean(axis=0))
    return centroids

def cohesion(data, labels):
    clusters = sorted(set(labels))
    sse = 0
    for cluster in clusters:
        cluster_data = data[labels == cluster, :]
        centroid = cluster_data.mean(axis = 0)
        sse += ((cluster_data - centroid)**2).sum()
    return sse

def separation(data, labels, cohesion_score):
    # calculate separation as SST - SSE
    return cohesion(data, np.zeros(data.shape[0])) - cohesion_score

def SST(data):
    c = get_centroids([data])
    return ((data - c) ** 2).sum()

def SSE(clusters, centroids):
    result = 0
    for cluster, centroid in zip(clusters, centroids):
        result += ((cluster - centroid) ** 2).sum()
    return result

# Clear the store before running each time
within_cluster_dist_sum_store = {}

# modifications added on 2018-12-23
# original
# def within_cluster_dist_sum(cluster, centroid, cluster_id):
# mod
def within_cluster_dist_sum(cluster, centroid, cluster_id, within_cluster_dist_sum_store):
    if cluster_id in within_cluster_dist_sum_store:
        return within_cluster_dist_sum_store[cluster_id]
    else:
        result = (((cluster - centroid) ** 2).sum(axis=1)**.5).sum()
        within_cluster_dist_sum_store[cluster_id] = result
    return result

def RMSSTD(data, clusters, centroids):
    df = data.shape[0] - len(clusters)
    attribute_num = data.shape[1]
    return (SSE(clusters, centroids) / (attribute_num * df)) ** .5

# equal to separation / (cohesion + separation)
def RS(data, clusters, centroids):
    sst = SST(data)
    sse = SSE(clusters, centroids)
    return (sst - sse) / sst

def DB_find_max_j(clusters, centroids, i):
    # Clear the store before running each time
    within_cluster_dist_sum_store = {} # modifications added on 2018-12-23
    
    max_val = 0
    max_j = 0
    for j in range(len(clusters)):
        if j == i:
            continue


        # modifications added on 2018-12-23
        # original
#         cluster_i_stat = within_cluster_dist_sum(clusters[i], centroids[i], i) / clusters[i].shape[0]
#         cluster_j_stat = within_cluster_dist_sum(clusters[j], centroids[j], j) / clusters[j].shape[0]
        # mod
        cluster_i_stat = within_cluster_dist_sum(clusters[i], centroids[i], i, within_cluster_dist_sum_store) / clusters[i].shape[0]
        cluster_j_stat = within_cluster_dist_sum(clusters[j], centroids[j], j, within_cluster_dist_sum_store) / clusters[j].shape[0]


        val = (cluster_i_stat + cluster_j_stat) / (((centroids[i] - centroids[j]) ** 2).sum() ** .5)
        if val > max_val:
            max_val = val
            max_j = j
    return max_val

def DB(data, clusters, centroids):
    result = 0
    for i in range(len(clusters)):
        result += DB_find_max_j(clusters, centroids, i)
    return result / len(clusters)

def XB(data, clusters, centroids):
    sse = SSE(clusters, centroids)
    min_dist = ((centroids[0] - centroids[1]) ** 2).sum()
    for centroid_i, centroid_j in list(product(centroids, centroids)):
        if (centroid_i - centroid_j).sum() == 0:
            continue
        dist = ((centroid_i - centroid_j) ** 2).sum()
        if dist < min_dist:
            min_dist = dist
    return sse / (data.shape[0] * min_dist)


In [1]:
def get_validation_scores(data, labels):
    # Clear the store before running each time
    within_cluster_dist_sum_store = {}

    within_cluster_dist_sum_store.clear()
    
    clusters = divide(data, labels)
    centroids = get_centroids(clusters)
    
    scores = {}

    scores['cohesion'] = cohesion(data, labels)
    scores['separation'] = separation(data, labels, scores['cohesion'])
    scores['calinski_harabaz_score'] = calinski_harabaz_score(data, labels)
    scores['RMSSTD'] = RMSSTD(data, clusters, centroids)
    scores['RS'] = RS(data, clusters, centroids)
    scores['DB'] = DB(data, clusters, centroids)
    scores['XB'] = XB(data, clusters, centroids)
    scores['silhouette_score'] = silhouette_score(data, labels)
    
    return scores
